In [1]:
import os 
import pandas as pd
import sys
import glob
import numpy as np 

In [2]:
#load bed file and counts file for B97 
bed=pd.read_csv('/global/scratch/users/chandlersutherland/e16/B97/genome/Zm-B97-REFERENCE-NAM-1_all_gene.bed', sep='\t', lineterminator='\n')
count=pd.read_csv('/global/scratch/users/chandlersutherland/e16/B97/rna_tip/STAR/ERR3791683_ReadsPerGene.out.tab', sep='\t', names=['name', 'unstranded', 'stranded_1', 'stranded_2'])
count=count.drop(labels=['unstranded', 'stranded_2'], axis=1)
#filter out summary stats and keep the column with 
count=count[~count['name'].str.contains('N_')]

In [3]:
count

,name,stranded_1
4,Zm00018ab000010,5
5,Zm00018ab000020,110
6,Zm00018ab000030,0
7,Zm00018ab000040,0
8,Zm00018ab000060,0
...,...,...
40367,Zm00018ab460410,0
40368,Zm00018ab460420,0
40369,Zm00018ab460430,0
40370,Zm00018ab460450,0


In [4]:
bed['gene_length'] = 1 + bed['chromEnd'] - bed['chromStart']
bed

,Unnamed: 0,chrom,chromStart,chromEnd,name,strand,gene_length
0,1,chr1,41909,47507,Zm00018ab000010,+,5599
1,78,chr1,47035,54079,Zm00018ab000020,-,7045
2,188,chr1,112990,117509,Zm00018ab000030,-,4520
3,251,chr1,118481,118840,Zm00018ab000040,+,360
4,255,chr1,190664,192763,Zm00018ab000060,-,2100
...,...,...,...,...,...,...,...
40363,1117805,scaf_727,33094,34203,Zm00018ab460410,-,1110
40364,1117812,scaf_755,210,2489,Zm00018ab460420,-,2280
40365,1117824,scaf_755,2980,3661,Zm00018ab460430,-,682
40366,1117833,scaf_797,26383,26877,Zm00018ab460450,+,495


In [5]:
#Merge counts output and gene length by gene name 
merged=pd.merge(bed, count, on='name')
#Calculate RPK by dividing readcount by length of gene in kb
merged['RPK']=merged['stranded_1']*1000/merged['gene_length']
#Calculate per million scaling factor for the sample 
#Count up all the RPK values in a sample and divide this number by 1,000,000. This is your “per million” scaling factor.
scaling=merged['RPK'].sum()/1000000
merged['TPM']=merged['RPK']/scaling
merged['log2(TPM)']=np.log2(merged['TPM']+1)
merged

,Unnamed: 0,chrom,chromStart,chromEnd,name,strand,gene_length,stranded_1,RPK,TPM,log2(TPM)
0,1,chr1,41909,47507,Zm00018ab000010,+,5599,5,0.893017,1.705367,1.435825
1,78,chr1,47035,54079,Zm00018ab000020,-,7045,110,15.613911,29.817424,4.945674
2,188,chr1,112990,117509,Zm00018ab000030,-,4520,0,0.000000,0.000000,0.000000
3,251,chr1,118481,118840,Zm00018ab000040,+,360,0,0.000000,0.000000,0.000000
4,255,chr1,190664,192763,Zm00018ab000060,-,2100,0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
40363,1117805,scaf_727,33094,34203,Zm00018ab460410,-,1110,0,0.000000,0.000000,0.000000
40364,1117812,scaf_755,210,2489,Zm00018ab460420,-,2280,0,0.000000,0.000000,0.000000
40365,1117824,scaf_755,2980,3661,Zm00018ab460430,-,682,0,0.000000,0.000000,0.000000
40366,1117833,scaf_797,26383,26877,Zm00018ab460450,+,495,0,0.000000,0.000000,0.000000


Ok, now repeat for all accessions and output to a single csv. This will require some finess to merge the correct tables together by accession and by rep. The final table will have: ecotype, rep, chrom, chromstart, chromend, name, strand, gene_length, count, tpm, log2(tpm). 

In [8]:
#get all bed files 
bed_files=glob.glob('/global/scratch/users/chandlersutherland/e16/*/genome/*all_gene.bed')

#get all counts files
count_files=glob.glob('/global/scratch/users/chandlersutherland/e16/*/rna_tip/STAR/*ReadsPerGene.out.tab')

52

In [26]:
#read in bed file and save accession and sample 
bed_df=pd.DataFrame()

for i in range(0, len(bed_files)):
    accession=bed_files[i].split('/')[6]
    bed=pd.read_csv(bed_files[i], sep='\t', lineterminator='\n')
    bed['accession']=accession
    bed['gene_length'] = 1 + bed['chromEnd'] - bed['chromStart']
    bed=bed.drop(['Unnamed: 0'], axis=1)
    
    bed_df=pd.concat([bed_df,bed])
    
bed_df

,chrom,chromStart,chromEnd,name,strand,accession,gene_length
0,chr1,41909,47507,Zm00018ab000010,+,B97,5599
1,chr1,47035,54079,Zm00018ab000020,-,B97,7045
2,chr1,112990,117509,Zm00018ab000030,-,B97,4520
3,chr1,118481,118840,Zm00018ab000040,+,B97,360
4,chr1,190664,192763,Zm00018ab000060,-,B97,2100
...,...,...,...,...,...,...,...
41054,scaf_1159,14194,18544,Zm00029ab462700,-,Ki3,4351
41055,scaf_1159,24174,24410,Zm00029ab462710,-,Ki3,237
41056,scaf_1159,24437,25012,Zm00029ab462720,-,Ki3,576
41057,scaf_1174,10087,10515,Zm00029ab462730,+,Ki3,429


In [33]:
#repeat with counts, with an extra column for biorep 
counts=pd.DataFrame()
for i in range(0, len(count_files)):
    accession=count_files[i].split('/')[6]
    sample=count_files[i].split('/')[-1].split('_')[0]
    
    #read in file 
    count=pd.read_csv(count_files[i], sep='\t', names=['name', 'unstranded', 'stranded_1', 'stranded_2'])
    count=count.drop(labels=['unstranded', 'stranded_2'], axis=1)
    
    #filter out summary stats and keep the column with 
    count=count[~count['name'].str.contains('N_')]
    
    #add accession and sample columns 
    count['accession']=accession
    count['sample']=sample 
    
    counts=pd.concat([counts,count])
    
counts

,name,stranded_1,accession,sample
4,Zm00018ab000010,3,B97,ERR3791685
5,Zm00018ab000020,130,B97,ERR3791685
6,Zm00018ab000030,0,B97,ERR3791685
7,Zm00018ab000040,0,B97,ERR3791685
8,Zm00018ab000060,0,B97,ERR3791685
...,...,...,...,...
41058,Zm00029ab462700,0,Ki3,ERR3791708
41059,Zm00029ab462710,0,Ki3,ERR3791708
41060,Zm00029ab462720,0,Ki3,ERR3791708
41061,Zm00029ab462730,0,Ki3,ERR3791708


In [40]:
#Merge counts output and gene length by gene name 
merged=pd.merge(bed_df, counts, on=['accession','name'])
#Calculate RPK by dividing readcount by length of gene in kb
merged['RPK']=merged['stranded_1']*1000/merged['gene_length']


In [81]:
#Calculate per million scaling factor for the sample 
#Count up all the RPK values in a sample and divide this number by 1,000,000. This is your “per million” scaling factor.
grouped=merged.groupby(['sample'])

tpm=pd.DataFrame()
for name, group in grouped: 
    scaling=group['RPK'].sum()/1000000
    group['TPM']=group['RPK']/scaling 
    tpm=pd.concat([tpm, group])

tpm

/global/software/sl-7.x86_64/modules/langs/python/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,chrom,chromStart,chromEnd,name,strand,accession,gene_length,stranded_1,sample,RPK,TPM
808153,chr1,34617,40204,Zm00001eb000010,+,B73,5588,10,ERR3773816,1.789549,2.630175
808155,chr1,41214,46762,Zm00001eb000020,-,B73,5549,2,ERR3773816,0.360425,0.529732
808157,chr1,108554,114382,Zm00001eb000050,-,B73,5829,0,ERR3773816,0.000000,0.000000
808159,chr1,188559,189581,Zm00001eb000060,-,B73,1023,19,ERR3773816,18.572825,27.297255
808161,chr1,190192,198832,Zm00001eb000070,-,B73,8641,0,ERR3773816,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
1786984,scaf_1508,8209,9582,Zm00042ab470600,+,Tzi8,1374,0,ERR3791734,0.000000,0.000000
1786986,scaf_1517,26315,28732,Zm00042ab470610,+,Tzi8,2418,0,ERR3791734,0.000000,0.000000
1786988,scaf_1546,1915,3135,Zm00042ab470620,-,Tzi8,1221,0,ERR3791734,0.000000,0.000000
1786990,scaf_1552,20729,22135,Zm00042ab470630,-,Tzi8,1407,0,ERR3791734,0.000000,0.000000


In [83]:
tpm['log2(TPM)']=np.log2(tpm['TPM']+1)
tpm=tpm[['accession', 'sample', 'name', 'chrom', 'chromStart', 'chromEnd', 'strand', 'gene_length', 'stranded_1', 'TPM', 'log2(TPM)']]
tpm
#write out to df 
tpm.to_csv('/global/scratch/users/chandlersutherland/e16/cs_reports/rna_tip_tpm.tsv', sep='\t')

/global/software/sl-7.x86_64/modules/langs/python/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [113]:
#NLR only 
gene_table='/global/home/users/chandlersutherland/e16/Maize_NLRome_GeneTable.txt'
gene=pd.read_csv(gene_table,sep = '\t')
nlrs=gene['Gene'].str.split('_', expand=True).iloc[:,0].str.replace('ZM', 'Zm').str.replace('AB', 'ab').unique()
gene['name'] = nlrs

nlr_tpm=tpm[tpm['name'].isin(nlrs)]
nlr_tpm.to_csv('/global/scratch/users/chandlersutherland/e16/cs_reports/rna_tip_tpm_nlr_only.tsv', sep='\t')

ValueError: Length of values does not match length of index

In [124]:
gene[gene.duplicated(subset=['Gene'])].sort_values(by='Gene')
#appears to be duplicated genes?

,Gene,Clade_0,Clade_1,Clade_2,Clade_3,Clade,Ecotype,Allele,File,HV
1602,ZM00001EB283200_P001,Int4977_21,Int4977_21,Int4916_25_31_R_18,NaN,Int4916_25_31_R_18,b73,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int4916...,0
1601,ZM00001EB283200_P001,Int4977_21,Int4916_25_31_R_18,Int4916_25_31_R_18,NaN,Int4916_25_31_R_18,b73,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int4916...,0
1512,ZM00001EB283200_P001,Int4916_25,Int4977_21,Int4916_25_31_R_18,NaN,Int4916_25_31_R_18,b73,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int4916...,0
700,ZM00001EB407930_P001,Int4084_115,Int4300_27,Int4084_115_153_R_26,NaN,Int4084_115_153_R_26,b73,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int4084...,0
828,ZM00001EB407930_P001,Int4300_27,Int4084_115_153_R_26,Int4084_115_153_R_26,NaN,Int4084_115_153_R_26,b73,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int4084...,0
...,...,...,...,...,...,...,...,...,...,...
1516,ZM00042AB296200_P001,Int4916_25,Int4977_21,Int4916_25_31_R_18,NaN,Int4916_25_31_R_18,tzi8,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int4916...,0
1581,ZM00042AB296200_P001,Int4977_21,Int4977_21,Int4916_25_31_R_18,NaN,Int4916_25_31_R_18,tzi8,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int4916...,0
831,ZM00042AB427440_P001,Int4300_27,Int4300_27,Int4084_115_153_R_26,NaN,Int4084_115_153_R_26,tzi8,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int4084...,0
830,ZM00042AB427440_P001,Int4300_27,Int4084_115_153_R_26,Int4084_115_153_R_26,NaN,Int4084_115_153_R_26,tzi8,1,RAxML_tree_pbNB-ARC/Clade_Refinement_1/Int4084...,0


In [126]:
sub=gene.drop_duplicates(subset=['Gene'])
sub['name']=nlrs

/global/software/sl-7.x86_64/modules/langs/python/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
